In [5]:
import gym
import torch
from agent import Agent, FloatTensor
import time
import tqdm as notebook_tqdm
env = gym.make('CartPole-v1')
## env_v1 = gym.make('CartPole-v1')

# input_dim,  output_dim, hidden_dim  are the same for v0 and v1, go dqn and ddqn
input_dim =  env.observation_space.shape[0] 
output_dim = env.action_space.n 
hidden_dim = 16
agent = Agent(input_dim, output_dim, hidden_dim)        
print('input_dim: ', input_dim, ', output_dim: ', output_dim, ', hidden_dim: ', hidden_dim)

input_dim:  4 , output_dim:  2 , hidden_dim:  16


In [6]:
def load(agent, directory, filename):
    agent.q_local.load_state_dict(
        torch.load('%s/%s_local.pth' % (directory,  filename)))
    agent.q_target.load_state_dict(
        torch.load('%s/%s_target.pth' % (directory,  filename)))

In [7]:
from collections import deque
import os
import numpy as np

def play(env, agent, n_episodes):
    state = env.reset()
    
    scores_deque = deque(maxlen=100)

    for i_episode in range(1, n_episodes+1):
        s = env.reset()        
        
        total_reward = 0
        time_start = time.time()
        timesteps =  0
        
        while True:
            
            a = agent.get_action(FloatTensor([s]), check_eps=False, eps=0.0)
            env.render()
            s2, r, done, _ = env.step(a.item())
            s = s2
            total_reward += r
            timesteps += 1
            
            if done:
                break 

        delta = (int)(time.time() - time_start)
        
        scores_deque.append(total_reward)

        print('Episode {}\tAverage Score: {:.2f}, \t Timesteps: {} \tTime: {:02}:{:02}:{:02}'\
                  .format(i_episode, np.mean(scores_deque), timesteps,\
                          delta//3600, delta%3600//60, delta%60))

In [8]:
load(agent, 'models', 'ddq')
play(env=env, agent=agent, n_episodes=5)
env.close()

Episode 1	Average Score: 179.00, 	 Timesteps: 179 	Time: 00:00:03
Episode 2	Average Score: 191.50, 	 Timesteps: 204 	Time: 00:00:04
Episode 3	Average Score: 189.33, 	 Timesteps: 185 	Time: 00:00:03
Episode 4	Average Score: 189.50, 	 Timesteps: 190 	Time: 00:00:03
Episode 5	Average Score: 186.60, 	 Timesteps: 175 	Time: 00:00:03
